# Reinforcement Learning with Expected Sarsa


### OpenAI Gym Taxi-v2
This is a Udacity Machine Learning Engineer NanoDegree Program mini-project.

This task was introduced in [Dietterich2000] to illustrate some issues in hierarchical reinforcement learning. There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

[Dietterich2000]	T Erez, Y Tassa, E Todorov, "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition", 2011.

source: https://gym.openai.com/envs/Taxi-v2/

### Some details

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt

import gym
env = gym.make("Taxi-v2")
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



- **Yellow rectangle** is the taxi when the taxi is empty and green with a passenger.
- **'I'** is a wall which the taxi can't cross.
- **R, G, Y, B** are the possible pick-up and drop-off locations. When the letter is blue, this is the pick-up location and then purple the drop-off.

- **State Space** size is 5x5x5x4 --> 500.
<br>(taxi rows, taxi colums, passenger state (4 locations + 1 in a the cab, 4 destinations)).
- **Action Space** size is 6:
    - 0 = south
    - 1 = north
    - 2 = east
    - 3 = west
    - 4 = pickup
    - 5 = dropoff

In [2]:
print("State Space:{}".format(env.observation_space))
print("Action Space:{}".format(env.action_space))

State Space:Discrete(500)
Action Space:Discrete(6)


### Agent

In [3]:
import numpy as np
from collections import defaultdict

class Agent:

    def __init__(self, nA=6):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.epsilon = 0.0001
        self.alpha = 0.2
        self.gamma = 0.9

    def update_Q(self, Qsa, Qsa_next, reward, alpha, gamma):
        """ updates the action-value function estimate using the most recent time step """
        return Qsa + (self.alpha * (reward + (self.gamma * Qsa_next) - Qsa))

    def epsilon_greedy_probs(self, Q_s, epsilon):
        """ obtains the action probabilities corresponding to epsilon-greedy policy """

        policy_s = np.ones(self.nA) * epsilon / self.nA
        policy_s[np.argmax(Q_s)] = 1 - epsilon + (epsilon / self.nA)
        return policy_s

    def select_action(self, state):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        state_policy = self.epsilon_greedy_probs(self.Q[state], self.epsilon)
        action = np.random.choice(np.arange(self.nA), p=state_policy)
        return action

    def step(self, state, action, reward, next_state, done):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # get epsilon-greedy action probabilities (for S')
        policy_s = self.epsilon_greedy_probs(self.Q[next_state], self.epsilon)

        # update Qsa

        self.Q[state][action] = self.update_Q(self.Q[state][action], np.dot(self.Q[next_state], policy_s), \
                                                  reward, self.alpha, self.gamma)

### Monitor

In [4]:
from collections import deque
import sys
import math
import time
from time import sleep
from IPython.display import clear_output

def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.

    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v1 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards and performance
    
    samp_rewards = deque(maxlen=window)
    
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        frames = []
        while True:
            # agent selects an action
            action = agent.select_action(state)
            # agent performs the selected action
            next_state, reward, done, _ = env.step(action)
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)
            # update the sampled reward
            samp_reward += reward
            # update the state (s <- s') to next time step
            state = next_state
            
            frames.append({'frame': env.render(mode='ansi'),
                           'state': state,
                           'action': action,
                           'reward': reward})
            
            if done:
                # save final sampled reward
                samp_rewards.append(samp_reward)
                
                break
        
        if (i_episode >= 100):
            
            """
            # monitor agent's real time movements
            for i, frame in enumerate(frames):
                clear_output(wait=True)
                print(frame['frame'])
                print(f"Timestep: {i + 1}")
                print(f"State: {frame['state']}")
                print(f"Action: {frame['action']}")
                print(f"Reward: {frame['reward']}")
                sleep(.1)
            """
           
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
            

               
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        
        
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')

    return avg_rewards, best_avg_reward


### OpenAI Gym - Taxi-v2

In [5]:
agent = Agent()
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 9.341

